<a href="https://colab.research.google.com/github/tedbach76/latticeflow/blob/main/threekings/fhp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import jax
import jax.numpy as jnp
import ipywidgets as widgets
from IPython.display import display
from PIL import Image
import io, os, time
import numpy as np

# --- CONFIGURATION (Golden Ratio Landscape 400x647) ---
GRID_SHAPE = (400, 647)
SNAPSHOT_DIR = "simpstep/hpp_snapshot"

class LatticeFlowHPP:
    def __init__(self, shape):
        self.shape = shape
        os.makedirs(SNAPSHOT_DIR, exist_ok=True)
        self.reset()

    def reset(self):
        self.t, self.stats_history, self.last_msps, self.forward = 0, [], 0.0, True
        key = jax.random.PRNGKey(int(time.time()))
        self.state = jax.random.bernoulli(key, p=0.5, shape=self.shape).astype(jnp.uint8)
        for i in range(1, 4):
            layer = jax.random.bernoulli(jax.random.fold_in(key, i), p=0.5, shape=self.shape).astype(jnp.uint8)
            self.state = self.state | (layer << i)
        self.state *= self._create_kings_mask(self.shape).astype(jnp.uint8)

    def _create_kings_mask(self, shape):
        mask = jnp.ones(shape, dtype=bool)

        def add_tri_crown(m, cx, yt):
            # Base bar
            m = m.at[yt+25:yt+32, cx-35:cx+35].set(0)
            # Three Triangular points using slopes
            for x in range(cx-35, cx+36):
                dist_l = abs(x - (cx-25))
                dist_m = abs(x - cx)
                dist_r = abs(x - (cx+25))
                # Left Point
                if yt + dist_l*1.5 < yt+25: m = m.at[int(yt + dist_l*1.5):yt+25, x].set(0)
                # Middle Point (Tallest)
                if yt-15 + dist_m*2.0 < yt+25: m = m.at[int(yt-15 + dist_m*2.0):yt+25, x].set(0)
                # Right Point
                if yt + dist_r*1.5 < yt+25: m = m.at[int(yt + dist_r*1.5):yt+25, x].set(0)
            return m

        # Centers for Golden Ratio width (647)
        cT, cM, cB = 160, 323, 486

        # --- T (and Crown) ---
        mask = add_tri_crown(mask, cT, 40)
        mask = mask.at[100:118, cT-50:cT+50].set(0)   # Top Bar
        mask = mask.at[118:320, cT-8:cT+8].set(0)     # Stem

        # --- M (and Crown) ---
        mask = add_tri_crown(mask, cM, 40)
        mask = mask.at[100:320, cM-45:cM-30].set(0)   # Leg L
        mask = mask.at[100:320, cM+30:cM+45].set(0)   # Leg R
        mask = mask.at[100:135, cM-30:cM+30].set(0)   # Solid Header
        mask = mask.at[135:210, cM-15:cM+15].set(0)   # Sharp V-Notch

        # --- B (and Crown) ---
        mask = add_tri_crown(mask, cB, 40)
        mask = mask.at[100:320, cB-40:cB-25].set(0)   # Spine
        mask = mask.at[100:115, cB-25:cB+25].set(0)   # Top
        mask = mask.at[200:215, cB-25:cB+25].set(0)   # Mid
        mask = mask.at[305:320, cB-25:cB+25].set(0)   # Bot
        mask = mask.at[100:320, cB+25:cB+40].set(0)   # Edge

        return mask

    @staticmethod
    @jax.jit
    def _hpp_rule(state):
        n, s, e, w = (state >> 0)&1, (state >> 1)&1, (state >> 2)&1, (state >> 3)&1
        coll_ns, coll_ew = (n & s) & ~(e | w), (e & w) & ~(n | s)
        new_n, new_s = jnp.where(coll_ns, 0, jnp.where(coll_ew, 1, n)), jnp.where(coll_ns, 0, jnp.where(coll_ew, 1, s))
        new_e, new_w = jnp.where(coll_ew, 0, jnp.where(coll_ns, 1, e)), jnp.where(coll_ew, 0, jnp.where(coll_ns, 1, w))
        return (new_n << 0) | (new_s << 1) | (new_e << 2) | (new_w << 3)

    @staticmethod
    @jax.jit
    def _forward_kernel(state):
        n, s, e, w = jnp.roll((state >> 0)&1, -1, 0), jnp.roll((state >> 1)&1, 1, 0), jnp.roll((state >> 2)&1, 1, 1), jnp.roll((state >> 3)&1, -1, 1)
        return LatticeFlowHPP._hpp_rule((n << 0) | (s << 1) | (e << 2) | (w << 3))

    @staticmethod
    @jax.jit
    def _backward_kernel(state):
        n, s, e, w = jnp.roll((state >> 0)&1, 1, 0), jnp.roll((state >> 1)&1, -1, 0), jnp.roll((state >> 2)&1, -1, 1), jnp.roll((state >> 3)&1, 1, 1)
        return LatticeFlowHPP._hpp_rule((n << 0) | (s << 1) | (e << 2) | (w << 3))

    def set_direction(self, fwd):
        if self.forward != fwd: self.forward = fwd; self.state = self._hpp_rule(self.state)

    def step(self):
        if self.forward: self.state = self._forward_kernel(self.state)
        else: self.state = self._backward_kernel(self.state)

class LatticeFlowUI:
    def __init__(self, sim):
        self.sim = sim
        self.text_in = widgets.Text(value='1000', description='Steps:', layout={'width': '140px'})
        self.render_in = widgets.Text(value='40', description='Render:', layout={'width': '140px'})
        self.btn_reset = widgets.Button(description="Reset", button_style='danger', layout={'width': '80px'})
        self.btn_snap = widgets.Button(description="Snapshot", button_style='success', layout={'width': '100px'})
        self.status = widgets.Label(value="Ready")
        self.header = widgets.HBox([self.text_in, self.render_in, self.btn_reset, self.btn_snap, self.status])
        self.stats_label = widgets.HTML(value="<b>T=0</b> | MSPS: 0.0")
        self.view = widgets.Image(format='png', width=700)
        self.text_in.on_submit(self.on_command); self.btn_reset.on_click(self.on_reset); self.btn_snap.on_click(self.on_snapshot)
        self.refresh_ui()

    def get_density_img(self, state):
        d = ((state >> 0) & 1) + ((state >> 1) & 1) + ((state >> 2) & 1) + ((state >> 3) & 1)
        pixels = (np.array(d) * 63).astype(np.uint8)
        img = Image.fromarray(pixels)
        buf = io.BytesIO(); img.save(buf, format='png'); return buf.getvalue()

    def refresh_ui(self, label="Updated"):
        img_bytes = self.get_density_img(self.sim.state)
        self.view.value = img_bytes
        if self.sim.stats_history:
            avg_v = sum(self.sim.stats_history) / len(self.sim.stats_history)
            self.stats_label.value = (f"<font color='#00ffcc' face='monospace'><b>T={self.sim.t}</b> | "
                                      f"MSPS: {self.sim.last_msps:.1f} (Avg: {avg_v:.1f}, "
                                      f"Max: {max(self.sim.stats_history):.1f})</font>")
        self.status.value = f"Status: {label}"

    def on_snapshot(self, b):
        fname = f"{SNAPSHOT_DIR}/tmb_echo_T{self.sim.t}_{int(time.time())}.png"
        with open(fname, "wb") as f: f.write(self.view.value)
        self.status.value = f"Snapshot saved: {fname}"

    def on_reset(self, b): self.sim.reset(); self.refresh_ui("Reset to T=0")

    def on_command(self, b):
        try: self.run_batch(int(self.text_in.value), int(self.render_in.value))
        except ValueError: self.status.value = "Status: Error"

    def run_batch(self, steps, render_freq):
        self.sim.set_direction(steps > 0); steps_abs = abs(steps)
        sites_per_step = self.sim.shape[0] * self.sim.shape[1]
        for i in range(0, steps_abs, render_freq):
            batch = min(render_freq, steps_abs - i)
            t0 = time.perf_counter()
            for _ in range(batch): self.sim.step(); self.sim.t += 1 if self.sim.forward else -1
            jax.block_until_ready(self.sim.state); t1 = time.perf_counter()
            self.sim.last_msps = (batch * sites_per_step) / ((t1 - t0) * 1e6)
            self.sim.stats_history.append(self.sim.last_msps)
            self.refresh_ui("Running...")
            time.sleep(0.01)
        self.refresh_ui("Batch Complete")

# --- EXECUTE ---
sim = LatticeFlowHPP(GRID_SHAPE)
ui = LatticeFlowUI(sim)
display(widgets.VBox([ui.header, ui.stats_label, ui.view]))

# LatticeFlow: The Three Kings Echo (HPP Edition)

## Executive Summary

LatticeFlow is a high-performance, bit-packed implementation of the **HPP (Hardy-Pomeau-Pazzis)** Lattice Gas Automaton. It serves as a modern bridge between the discrete cellular automata hardware of the 1990s (**CAM-8**) and modern tensor computing. This experiment demonstrates the **"Three Kings Echo"**—a bit-perfect restoration of information following total thermalization, proving that entropy in a closed, reversible system is a matter of perspective rather than information loss.

### **The Methodology**

The LatticeFlow engine achieves its efficiency and reversibility by strictly separating the two physical components of a lattice gas:

1. **Transport (The Shift):** A non-local operation where data bit-planes are rolled across the grid based on velocity vectors.
2. **Interaction (The Rule):** A strictly local operation where HPP collision logic ($N+S \leftrightarrow E+W$) is applied at each site to conserve mass and momentum.

This separation allows for a perfectly reversible evolution. By applying a "Phase-Sync" bridge—executing an interaction rule between the forward and backward transport phases—the system can backtrack through its own history with zero noise.

### **Hardware & Performance Validation**

The benchmark results were obtained on a standardized cloud environment with high-capacity L3 cache, facilitating full lattice residency.

* **Processor:** Intel(R) Xeon(R) @ 2.20GHz (Broadwell, Family 6, Model 79)
* **Architecture:** x86_64, 2 Cores, 55MB L3 Cache
* **Grid Size:** 400 x 647 (Golden Ratio Landscape)
* **Peak Throughput:** **1470.5 MSPS** (Mega-Sites Per Second)
* **Sustained Performance:** **1367.7 MSPS**

This performance—reaching **1.47 Billion updates per second** on a dual-core CPU—is achieved through bit-packing 4-bit HPP states into `uint8` tensors and leveraging XLA (Accelerated Linear Algebra) to fuse bitwise kernels into optimal machine instructions.

### **The Three Kings Experiment**

1. **Initial State ($T=0$):** The initials of "Three Kings" of reversible computing—**Toffoli (T), Margolus (M), and Bennett (B)**—are prepared with three-point regal crowns, centered according to the golden ratio.
2. **Forward Evolution:** The system is run forward (e.g., 200 steps), causing the structured initials and crowns to dissolve into a high-entropy gas.
3. **The Echo:** By applying the time-reversal operator (reversing transport and syncing the collision phase), the system backtracks.
4. **Restoration:** The gas re-coalesces into the original, pixel-perfect initials and crowns, demonstrating that underlying correlations are preserved even in a chaotic state.

### **Lineage & Contributions**

This project represents a generative physics collaboration, echoing the foundational work of **Tommaso Toffoli, Norman Margolus, and Charles Bennett**. The modern LatticeFlow implementation and benchmarking were developed by **Edward "Ted" A. Bach**, utilizing a "Perplexity-Driven" collaborative loop with generative AI to synthesize optimized physical kernels.

All experimental results and performance benchmarks can be archived using the integrated **Snapshot** utility, which saves timestamped PNGs and state data to the `simpstep/hpp_snapshot/` directory.

See also this blog post https://tedbach76.github.io/latticeflow/threekings/